## **0. Introduction**

**NOTE: You should use GPU if you want to train this model**


- This notebook is about **ESPCN** (Efficient Sub-Pixel Convolutional Neural Network), and **how it is applied into SISR** **bold text** (Single Image Super Resolution) problem

- 3 parts:
  - **Data Preperation** - get the train and test datasets
  - **Testing / Evaluate** - use the weight TRAINED BY US from scratch to evaluate
  - **Try training** - help you see how we train this model

## **1. Data Preperation**

In [1]:
# import necessary libs
!pip install datasets==2.15 super-image
from datasets import load_dataset
from super_image.data import EvalDataset, EvalMetrics, TrainDataset
from IPython.display import clear_output # only clear the output to have better look
clear_output()

In [2]:
# Dataset for testing (Set5, Set14, BSDS100)
set5 = load_dataset('eugenesiow/Set5', "bicubic_x4", split='validation')    # 5 images
set14 = load_dataset('eugenesiow/Set14', "bicubic_x4", split='validation')  # 14 images
b100 = load_dataset('eugenesiow/BSD100', "bicubic_x4", split='validation')  # 100 images

eval_set5 = EvalDataset(set5)
eval_set14 = EvalDataset(set14)
eval_b100 = EvalDataset(b100)
clear_output()

In [3]:
# Dataset for training (DIV2K)
## it takes nearly 4 minutes to download
## Only run this if you want to TRY TRAINING MODEL
div2k = load_dataset('eugenesiow/Div2k', 'bicubic_x4', split='train')   # 800 images
train_div2k = TrainDataset(div2k)
clear_output()

## **2. Testing / Evaluate**

- We will use the weight that was **TRAIN BY US** before to **evaluate**

### **2.1. Define the model**

In [4]:
# import necessary libs
import torch
from torch import nn

In [5]:
# Define the model
class ESPCN(nn.Module):
    def __init__(self):
        super().__init__()

        # Feature mapping
        self.feature_maps = nn.Sequential(
            nn.Conv2d(3, 64, (5, 5), (1, 1), (2, 2)),
            nn.Tanh(),
            nn.Conv2d(64, 32, (3, 3), (1, 1), (1, 1)),
            nn.Tanh(),
        )

        # Sub-pixel convolution layer
        self.sub_pixel = nn.Sequential(
            nn.Conv2d(32, 3*16, (3, 3), (1, 1), (1, 1)),
            nn.PixelShuffle(4),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.feature_maps(x)
        x = self.sub_pixel(x)
        x = torch.clamp_(x, 0.0, 1.0)
        return x

### **2.2. Download and load the weight**

In [6]:
# Download the weight

import requests
url = "https://raw.githubusercontent.com/JerryHVK/DNSM/main/espcn_div2k_x4.pth"
# this is the url to my github repo, where i put the model weight that trained by myself

response = requests.get(url)

if response.status_code == 200:
    with open("espcn_div2k_x4.pth", "wb") as file:
        file.write(response.content)
    print("File downloaded successfully")
else:
    print(f"Failed to download file. Status code: {response.status_code}")



File downloaded successfully


In [7]:
# Load the downloaded weight
espcn_model = ESPCN()
model_path = "/content/espcn_div2k_x4.pth"
espcn_model.load_state_dict(torch.load(f = model_path, map_location=torch.device("cpu")))

<All keys matched successfully>

### **2.3. Run the evaluation**

In [8]:
# Evaluate the model using PSNR (Peak Signal to Noise Ration) and SSIM

print("ESPCN on Set5")
EvalMetrics().evaluate(espcn_model, eval_set5)
print("ESPCN on Set14")
EvalMetrics().evaluate(espcn_model, eval_set14)
print("ESPCN on BSD100")
EvalMetrics().evaluate(espcn_model, eval_b100)

ESPCN on Set5


Evaluating dataset:   0%|          | 0/5 [00:00<?, ?it/s]

scale:4      eval psnr: 29.33     ssim: 0.8254
ESPCN on Set14


Evaluating dataset:   0%|          | 0/14 [00:00<?, ?it/s]

scale:4      eval psnr: 26.83     ssim: 0.7354
ESPCN on BSD100


Evaluating dataset:   0%|          | 0/100 [00:00<?, ?it/s]

scale:4      eval psnr: 27.22     ssim: 0.7246


## **3. Try training**

- You will experience a part of what we have done, using PyTorch Framework

### **3.1. Making Dataloader (to load data for training)**

In [9]:
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
train_dataloader = DataLoader(train_div2k, batch_size=1, shuffle=True)

### **3.2. Define the loss function, optimizer**

In [10]:
torch.manual_seed(106)
try_model = ESPCN()

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(params= try_model.parameters(), lr = 0.0003)

### **3.3. Define the train function**

In [11]:
# Define the training step
def train_step(model: torch.nn.Module,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               dataloader: torch.utils.data.DataLoader,
               device: torch.device):
  """Perform training step and return the training loss"""
  train_loss = 0
  model.train()
  for lr, hr in tqdm(dataloader):
    lr, hr = lr.to(device), hr.to(device)
    # 1. Forward pass
    hr_pred = model(lr)
    # 2. Calculate the loss
    loss = loss_fn(hr_pred, hr)
    train_loss += loss.detach()
    # train_loss += loss.cpu().detach().item()
    # 3. Optimizer zero grad
    optimizer.zero_grad()
    # 4. backward
    loss.backward()
    # 5. optimizer step
    optimizer.step()
  train_loss = train_loss / len(dataloader)
  return train_loss

In [15]:
def train(epochs: int, device: torch.device):
  try_model.to(device)
  for epoch in tqdm(range(epochs)):
    train_loss = train_step(model = try_model,
                            loss_fn = loss_fn,
                            optimizer = optimizer,
                            dataloader = train_dataloader,
                            device = device)

    print(f"Epoch: {epoch} | train_loss = {train_loss: .7f}")

### **3.4. Start training**

In [16]:
# Choose the device to train
device = "cuda" if torch.cuda.is_available() else "cpu"
if(device == "cpu"):
  print("You will using CPU to train this model, and it will be very slow")
  print("Consider to use a GPU")

In [17]:
# Declare the number of epochs to train model
# You need to train 30 epochs to have the weight as good as ours
epochs = 2
# Start training
train(epochs = epochs, device = device)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/800 [00:00<?, ?it/s]

Epoch: 0 | train_loss =  0.0161512


  0%|          | 0/800 [00:00<?, ?it/s]

Epoch: 1 | train_loss =  0.0055431


#### **Thanks for reviewing our results !!!**